# 自動機械学習を使用する

モデルのトレーニングに使用できる機械学習アルゴリズムにはさまざまな種類があり、特定のデータや予測要件に最も効果的なアルゴリズムを特定するのは容易ではない場合があります。さらに、正規化、欠落している機能の代入などの手法を使用して、トレーニング データを前処理することによってモデルの予測パフォーマンスに大きな影響を与えることができます。要件に*最適*なモデルを見つけるには、多くのアルゴリズムと前処理変換の組み合わせを試す必要があります。これは多くの時間とコンピューティング リソースを必要とします。

Azure Machine Learning により、さまざまなアルゴリズムと前処理オプションを使用して、トレーニング済みモデルの比較を自動化できます。[Azure Machine Learning Studio](https://ml/azure.com) または SDK のビジュアル インターフェイスを使って、この機能を活用することができます。SDK では、自動機械学習実験の設定をより細かくコントロールできますが、ビジュアル インターフェイスはより使いやすくなっています。

## 開始する前に

このノートブックのコードを実行するには、最新バージョンの **azureml-sdk** および **azureml-widgets** パッケージに加えて、**azureml-train-automl** パッケージが必要です。次のセルを実行して、パッケージがインストールされていることを確認します。

In [ ]:
!pip show azureml-train-automl

## ワークスペースに接続する

必要な SDK パッケージがインストールされているため、ワークスペースに接続できます。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存された構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## データを準備する

自動機械学習用トレーニング スクリプトを作成する必要はありませんが、トレーニング データを読み込む必要があります。この場合、糖尿病患者の詳細を含むデータセットを使用し、これをトレーニング用とモデル検証用の 2 つのデータセットに分割します。

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')


# トレーニングと検証のサブセットにデータセットを分割する
diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

## コンピューティングを準備する

クラウド コンピューティングの利点の 1 つは、オンデマンドでスケーリングできることです。これにより、自動機械学習実験の複数の子実行を並行して処理するのに十分なコンピューティング リソースをプロビジョニングできます。

次のコードを使用して Azure Machine Learning コンピューティング クラスター (存在しない場合は作成されます) を指定します。

> **重要**: 実行する前に、以下のコードで *your-compute-cluster* をコンピューティング クラスターの名前に変更してください。クラスター名は、長さが 2 〜 16 文字のグローバルに一意の名前である必要があります。英字、数字、- の文字が有効です。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **注**: コンピューティング インスタンスとクラスターは、スタンダードの Azure 仮想マシンのイメージに基づいています。この演習では、コストとパフォーマンスの最適なバランスを実現するために、*Standard_DS11_v2* イメージが推薦されます。サブスクリプションにこのイメージを含まないクォータがある場合は、別のイメージを選択してください。 ただし、画像が大きいほどコストが高くなり、小さすぎるとタスクが完了できない場合があることに注意してください。Azure 管理者にクォータを拡張するように依頼していただくことも可能です。

## 自働機械学習を構成する

これで、自動機械学習実験を構成する準備が整いました。

最も重要な構成設定の 1 つは、モデルのパフォーマンスを評価するメトリックです。次のように、特定のタイプのモデルタスク (分類または回帰) の自動機械学習によって計算されるメトリックのリストを取得できます。

In [ ]:
import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('classification'):
    print(metric)

最適化するメトリック (この例では *AUC_weighted*) を決定したら、自動機械学習の実行を構成できます。そのためには、使用するデータや試行する組み合わせの数などのオプションとともに、ターゲット メトリックを指定する AutoML 構成が必要です。

> **注**: この例では、実験を 4 回に制限して、所要時間を短縮します。実際には、さらに多くの反復を試みる可能性があります。

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='Diabetic',
                             iterations=4,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=2,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

## 自動機械学習の実験を実行する

準備完了です。自動機械学習実験を実行しましょう。

> **注**: これには時間がかかる場合があります。

In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'mslearn-diabetes-automl-sdk')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## 子実行の詳細を表示する

実験が完了したら、ウィジェットで出力を表示し、最適な結果を生成した実行をクリックして詳細を表示します。

次に、Azure Machine Learning Studio で実験の詳細を表示し、実験の詳細を表示してから、最良の結果を得た個々の実行の詳細を表示するリンクをクリックします。ここには、生成されたモデルのパフォーマンスに関する情報が多数あります。

また、すべての子実行を取得し、SDK を使用してメトリックを表示することもできます。

In [ ]:
for run in automl_run.get_children():
    print('Run ID', run.id)
    for metric in run.get_metrics():
        print('\t', run.get_metrics(metric))

## 最適なパフォーマンスを得る

最高のパフォーマンスを取得し、その詳細を次のように確認できます。

In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

最後に、最もパフォーマンスの高いモデルを見つけたので、それを登録できます。

In [ ]:
from azureml.core import Model

# モデルを登録する
best_run.register_model(model_path='outputs/model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# 登録済みモデルを一覧表示する
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳細情報**: 自動機械学習の詳細については、[Azure ML のドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train)を参照してください。